# Deliverable 2. Create a Customer Travel Destinations Map.
----
1. Create a folder called `Vacation_Search` to save all the files related with this deliverable.

2. Download the `Vacation_Search_starter_code.ipynb` Jupyter notebook,save it into your `Vacation_Search folder`, and rename it as `Vacation_Search.ipynb`.

3. In the `Vacation_Search.ipynb` file, ensure that the dependencies and the Google API key is imported correctly.

4. From the `Weather_Database` folder you created in the "Deliverable 1," import the `WeatherPy_Database.csv` file as a Pandas DataFrame named `city_data_df`.

5. Write two input statements that prompt the user to enter their minimum and maximum temperature criteria for their vacation.

6. Create a new Pandas DataFrame by using the `loc` Pandas method to filter the `city_data_df` DataFrame for temperature criteria  collected. Name the DataFrame as `preferred_cities_df`.

7. Create a new Pandas DataFrame named `clean_travel_cities` by using the Pandas `dropna` function on the `preferred_cities_df` to drop any empty rows.

8. Use the `copy` Pandas function to create a new DataFrame, called `hotel_df`, by copying the following columns from the `clean_travel_cities` DataFrame: "City", "Country", "Max Temp", "Current Description", "Lat", "Lng".

9. Add a new empty column named `Hotel Name` to the `hotel_df` DataFrame.

10. Review the hotel search parameters provided. These parameters are the same we used in this module; you'll use them to search for a hotel for each city.

11. Use a for loop to iterate through the `hotel_df` DataFrame, retrieve the latitude and longitude of each city to find the nearest hotel based on the search parameters provided, then add the hotel name to the `hotel_df` DataFrame. If a hotel isn't found, skip to the next city.

12. Drop any rows in the `hotel_df` DataFrame where a hotel name is not found and store the resulting data into a new DataFrame named `clean_hotel_df`.

13. Create an CSV file to store the `clean_hotel_df` DataFrame as `WeatherPy_vacation.csv` in the Vacation_Search folder.

14. Review the formatting template provided that you'll use to add an information box to each marker in the map. In the pop-up for each city you'll add:

    1. The city name.

    2. The country code.

    3. The weather description and maximum temperature for the city.

15. Review the provided list comprehension code to retrieve the city data from each row, which will then be added to the formatting template and saved in the `hotel_info` list.

16. Use the provided code snippet to retrieve the latitude and longitude from each row and store them in a new DataFrame called `locations`.

17. Refactor your previous marker layer map code to create a marker layer map that will have pop-up markers for each city on the map.

18. Take a screenshot of your map and save it to the `Vacation_Search` folder as `WeatherPy_vacation_map.png`.
---

## Ensure that the dependencies and the Google API key is imported correctly.

In [1]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps
import os
import collections.abc 

# Import Google API key
from config import g_key

# Configure gmaps
gmaps.configure(api_key=g_key)

## From the `Weather_Database` folder you created in the "Deliverable 1," import the `WeatherPy_Database.csv` file as a Pandas DataFrame.

In [2]:
# Set the file path to import the WeatherPy_database.csv file

file_path = "../Weather_Database/WeatherPy_Database.csv"

# Load the CSV file into a Pandas DataFrame

city_data_df = pd.read_csv(os.path.join("../Weather_Database/WeatherPy_Database.csv"))

# Display sample data

city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Laguna,US,38.4210,-121.4238,66.49,36,0,3.44,clear sky
1,1,Hirara,JP,24.8000,125.2833,75.47,94,20,3.44,few clouds
2,2,San Andres,CO,12.5847,-81.7006,80.62,89,75,10.36,broken clouds
3,3,Qaanaaq,GL,77.4840,-69.3632,-16.98,74,3,7.72,clear sky
4,4,Windhoek,NaN,-22.5594,17.0832,69.42,45,32,4.61,scattered clouds


## Write two input statements that prompt the user to enter their minimum and maximum temperature criteria for their vacation.

In [3]:
# Prompt the user to enter the minimum temperature criteria

min_temp = float(input("What is the minimun temperature you would like for your trip? "))

# Prompt the user to enter the maximum temperature criteria

max_temp = float(input("What is the maximun temperature you would like for your trip? "))

What is the minimun temperature you would like for your trip? 70
What is the maximun temperature you would like for your trip? 95


## Create a new Pandas DataFrame by using the `loc` Pandas method to filter the `city_data_df` DataFrame for temperature criteria  collected.

In [4]:
# Filter the city_data_df DataFrame to find the cities that fit the criteria using the loc Pandas function

preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] >= min_temp) & (city_data_df["Max Temp"] <= max_temp)]
                                    
# Display sample data

preferred_cities_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
1,1,Hirara,JP,24.8000,125.2833,75.47,94,20,3.44,few clouds
2,2,San Andres,CO,12.5847,-81.7006,80.62,89,75,10.36,broken clouds
11,11,Boquim,BR,-11.1469,-37.6206,71.67,92,17,5.66,few clouds
12,12,Puerto Ayacucho,VE,5.6639,-67.6236,74.97,92,100,3.18,overcast clouds
13,13,Victoria,HK,22.2855,114.1577,75.38,87,0,5.01,clear sky


## Create a new Pandas DataFrame named `clean_travel_cities` by using the Pandas `dropna` function on the `preferred_cities_df` to drop any empty rows.

In [5]:
# Drop any empty rows in the preferred_cities_df DataFrame and create a new DataFrame.

clean_travel_cities_df = preferred_cities_df.dropna(axis="index", how="any")

# Display sample data

clean_travel_cities_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
1,1,Hirara,JP,24.8000,125.2833,75.47,94,20,3.44,few clouds
2,2,San Andres,CO,12.5847,-81.7006,80.62,89,75,10.36,broken clouds
11,11,Boquim,BR,-11.1469,-37.6206,71.67,92,17,5.66,few clouds
12,12,Puerto Ayacucho,VE,5.6639,-67.6236,74.97,92,100,3.18,overcast clouds
13,13,Victoria,HK,22.2855,114.1577,75.38,87,0,5.01,clear sky


## Use the `copy` Pandas function to create a new DataFrame, called `hotel_df`, by copying the following columns from the `clean_travel_cities` DataFrame: "City", "Country", "Max Temp", "Current Description", "Lat", "Lng".

In [6]:
# Create DataFrame called hotel_df by copying some columns from the clean_travel_cities DataFrame.

hotel_df = clean_travel_cities_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# Display sample data

hotel_df.head()

,City,Country,Max Temp,Current Description,Lat,Lng
1,Hirara,JP,75.47,few clouds,24.8000,125.2833
2,San Andres,CO,80.62,broken clouds,12.5847,-81.7006
11,Boquim,BR,71.67,few clouds,-11.1469,-37.6206
12,Puerto Ayacucho,VE,74.97,overcast clouds,5.6639,-67.6236
13,Victoria,HK,75.38,clear sky,22.2855,114.1577


## Add a new empty column named `Hotel Name` to the `hotel_df` DataFrame.

In [7]:
# Add a new empty column, "Hotel Name", to the hotel_df DataFrame

hotel_df["Hotel Name"] = ""

# Display sample data
hotel_df.head()

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
1,Hirara,JP,75.47,few clouds,24.8000,125.2833,
2,San Andres,CO,80.62,broken clouds,12.5847,-81.7006,
11,Boquim,BR,71.67,few clouds,-11.1469,-37.6206,
12,Puerto Ayacucho,VE,74.97,overcast clouds,5.6639,-67.6236,
13,Victoria,HK,75.38,clear sky,22.2855,114.1577,


## Review the hotel search parameters provided. These parameters are the same we used in this module; you'll use them to search for a hotel for each city.

In [8]:
# Review the parameters to search for a hotel
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}        

## Use a for loop to iterate through the `hotel_df` DataFrame, retrieve the latitude and longitude of each city to find the nearest hotel based on the search parameters provided, then add the hotel name to the `hotel_df` DataFrame. If a hotel isn't found, skip to the next city.

In [9]:
# Iterate through the hotel DataFrame 

for index, row in hotel_df.iterrows():
    
    # Fetch latitude and longitude from the DataFrame
    
    lat = row["Lat"]
    
    lng = row["Lng"]
    
    # Add the latitude and longitude as parameters to the params dictionary
    
    params["location"] = f"{lat},{lng}"
    
    # Set up the base URL for the Google Directions API to get JSON data
    
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # Make an API request and retrieve the JSON data from the hotel search
    
    hotels = requests.get(base_url, params=params).json()
    
    # Get the first hotel from the results and store the name, if a hotel isn't found skip the city
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


## Drop any rows in the `hotel_df` DataFrame where a hotel name is not found and store the resulting data into a new DataFrame named `clean_hotel_df`.

In [10]:
# Drop the rows where there is no Hotel Name.

clean_hotel_df = hotel_df.dropna(axis="index", how="any")

# Display sample data

clean_hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
1,Hirara,JP,75.47,few clouds,24.8000,125.2833,Hotel Atoll Emerald Miyakojima
2,San Andres,CO,80.62,broken clouds,12.5847,-81.7006,Hotel Arena Blanca
11,Boquim,BR,71.67,few clouds,-11.1469,-37.6206,Moda vip BOQUIM
12,Puerto Ayacucho,VE,74.97,overcast clouds,5.6639,-67.6236,Posada Turistica Manapiare
13,Victoria,HK,75.38,clear sky,22.2855,114.1577,Mini Central
17,Rikitea,PF,71.87,light rain,-23.1203,-134.9692,People ThankYou
19,Vaini,TO,82.56,few clouds,-21.2000,-175.2000,Keleti Beach Resort
21,Atuona,PF,78.17,broken clouds,-9.8000,-139.0333,Villa Enata
27,Richards Bay,ZA,71.64,light rain,-28.7830,32.0377,BON Hotel Waterfront Richards Bay
32,Kapaa,US,82.72,scattered clouds,22.0752,-159.3190,Sheraton Kauai Coconut Beach Resort


**Note:** If the resulting DataFrame is empty, uncomment the following code to load sample data into the `clean_hotel_df`.

In [11]:
# In case of an empty DataFrame, load the sample data provided
# clean_hotel_df = pd.read_csv("WeatherPy_vacation-backup.csv")
# clean_hotel_df.head(10)

## Create an CSV file to store the `clean_hotel_df` DataFrame as `WeatherPy_vacation.csv` in the Vacation_Search folder.

In [12]:
# Set the file name.

output_data_file = "WeatherPy_vacation.csv"

# Create a CSV file by using the clean_hotel_df DataFrame

clean_hotel_df.to_csv(output_data_file, index_label="City_ID")
clean_hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
1,Hirara,JP,75.47,few clouds,24.8000,125.2833,Hotel Atoll Emerald Miyakojima
2,San Andres,CO,80.62,broken clouds,12.5847,-81.7006,Hotel Arena Blanca
11,Boquim,BR,71.67,few clouds,-11.1469,-37.6206,Moda vip BOQUIM
12,Puerto Ayacucho,VE,74.97,overcast clouds,5.6639,-67.6236,Posada Turistica Manapiare
13,Victoria,HK,75.38,clear sky,22.2855,114.1577,Mini Central
17,Rikitea,PF,71.87,light rain,-23.1203,-134.9692,People ThankYou
19,Vaini,TO,82.56,few clouds,-21.2000,-175.2000,Keleti Beach Resort
21,Atuona,PF,78.17,broken clouds,-9.8000,-139.0333,Villa Enata
27,Richards Bay,ZA,71.64,light rain,-28.7830,32.0377,BON Hotel Waterfront Richards Bay
32,Kapaa,US,82.72,scattered clouds,22.0752,-159.3190,Sheraton Kauai Coconut Beach Resort


### Review the formatting template provided that you'll use to add an information box to each marker in the map.

In the pop-up for each city you'll add: 

1. The city name

2. The country code

3. The weather description and maximum temperature for the city

In [13]:
# Review the formatting template provided

info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Weather</dt><dd>{Current Description} and {Max Temp} °F</dd>
</dl>
"""

## Review the provided list comprehension code to retrieve the city data from each row, which will then be added to the formatting template and saved in the `hotel_info` list.

In [14]:
# Get the data from each row in the clean_hotel_df DataFrame, add it to the formatting template, and store the data in a list.

hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# Display sample data

hotel_info[:10]

['\n<dl>\n<dt>Hotel Name</dt><dd>Hotel Atoll Emerald Miyakojima</dd>\n<dt>City</dt><dd>Hirara</dd>\n<dt>Country</dt><dd>JP</dd>\n<dt>Current Weather</dt><dd>few clouds and 75.47 °F</dd>\n</dl>\n',
 '\n<dl>\n<dt>Hotel Name</dt><dd>Hotel Arena Blanca</dd>\n<dt>City</dt><dd>San Andres</dd>\n<dt>Country</dt><dd>CO</dd>\n<dt>Current Weather</dt><dd>broken clouds and 80.62 °F</dd>\n</dl>\n',
 '\n<dl>\n<dt>Hotel Name</dt><dd>Moda vip BOQUIM</dd>\n<dt>City</dt><dd>Boquim</dd>\n<dt>Country</dt><dd>BR</dd>\n<dt>Current Weather</dt><dd>few clouds and 71.67 °F</dd>\n</dl>\n',
 '\n<dl>\n<dt>Hotel Name</dt><dd>Posada Turistica Manapiare</dd>\n<dt>City</dt><dd>Puerto Ayacucho</dd>\n<dt>Country</dt><dd>VE</dd>\n<dt>Current Weather</dt><dd>overcast clouds and 74.97 °F</dd>\n</dl>\n',
 '\n<dl>\n<dt>Hotel Name</dt><dd>Mini Central</dd>\n<dt>City</dt><dd>Victoria</dd>\n<dt>Country</dt><dd>HK</dd>\n<dt>Current Weather</dt><dd>clear sky and 75.38 °F</dd>\n</dl>\n',
 '\n<dl>\n<dt>Hotel Name</dt><dd>People Th

## Use the provided code snippet to retrieve the latitude and longitude from each row and store them in a new DataFrame called `locations`.

In [15]:
# Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

# Display sample data
locations.head(10)

,Lat,Lng
1,24.8000,125.2833
2,12.5847,-81.7006
11,-11.1469,-37.6206
12,5.6639,-67.6236
13,22.2855,114.1577
17,-23.1203,-134.9692
19,-21.2000,-175.2000
21,-9.8000,-139.0333
27,-28.7830,32.0377
32,22.0752,-159.3190


## Refactor your previous marker layer map code to create a marker layer map that will have pop-up markers for each city on the map.

In [16]:
# Add a marker layer for each city to the map. 

marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)

# Create a figure to add the Google map as a layer

fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)

fig.add_layer(marker_layer)

# Display the figure containing the map

fig

AttributeError: module 'collections' has no attribute 'Iterable'